In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# ---------------- SETTINGS ----------------
DATA_DIR = r"F:\Work\ORANGE MODELS\brain_tumor_dataset\train"
MODEL_PATH = "brain_ct_classifier.pth"
BATCH_SIZE = 32
IMG_SIZE = 224
EPOCHS = 5
# ------------------------------------------

# Transform (resize + normalize)
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])  # standard ImageNet norm
])

# Dataset + DataLoader
train_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Get class names
class_names = train_dataset.classes
print("Classes:", class_names)

# Simple model: Pretrained ResNet18
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(class_names))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss + Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    running_loss, correct = 0.0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct / len(train_dataset) * 100
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.2f}%")

# Save model
torch.save({
    "model_state": model.state_dict(),
    "class_names": class_names
}, MODEL_PATH)

print("✅ Model saved at", MODEL_PATH)


Classes: ['Healthy', 'Tumor']


c:\Users\hosny\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hosny\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5 - Loss: 0.0786, Acc: 96.98%
Epoch 2/5 - Loss: 0.0247, Acc: 99.23%
Epoch 3/5 - Loss: 0.0070, Acc: 99.76%
Epoch 4/5 - Loss: 0.0102, Acc: 99.68%
Epoch 5/5 - Loss: 0.0057, Acc: 99.83%
✅ Model saved at brain_ct_classifier.pth


In [8]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# ---------------- SETTINGS ----------------
MODEL_PATH = r"F:\Work\ORANGE MODELS\brain_tumor_dataset\train\brain_ct_classifier.pth"
IMG_PATH = r"F:\Work\ORANGE MODELS\brain_tumor_dataset\val\Healthy\ct_healthy (3).png"
IMG_SIZE = 224
# ------------------------------------------

# Load checkpoint
checkpoint = torch.load(MODEL_PATH, map_location="cpu")
class_names = checkpoint["class_names"]

# Rebuild model
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model.load_state_dict(checkpoint["model_state"])
model.eval()

# Transform
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load image
img = Image.open(IMG_PATH).convert("RGB")
img = transform(img).unsqueeze(0)  # add batch

# Predict
with torch.no_grad():
    outputs = model(img)
    probs = torch.softmax(outputs, dim=1)[0]
    pred_idx = torch.argmax(probs).item()
    confidence = probs[pred_idx].item()

print("\n--- Prediction ---")
for i, p in enumerate(probs):
    print(f"{class_names[i]}: {p*100:.2f}%")

print(f"\n➡️ Final Prediction: {class_names[pred_idx]} (confidence: {confidence*100:.2f}%)")



--- Prediction ---
Healthy: 99.92%
Tumor: 0.08%

➡️ Final Prediction: Healthy (confidence: 99.92%)
